In [2]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from typing import Any
from scipy.interpolate import interp1d
import numpy as np
import copy

from src.util.enumerations import Economy, get_economic_scenario

In [ ]:
class MatterDefaults:
    def __init__(self):
        self.recycling_rate = 0.27  # Example default value
        self.proportion_of_discarded_stock = 0.013
        self.conversion_rate_material_reserves = 0.0015

In [ ]:


class MatterUse:
    """
    This class describes the matter-use dynamics in the JUSTICE model.
    """
    def __init__(
        self, 
        input_dataset,
        time_horizon,
        scenario
        ):

        #Create an instance of MatterDefaults
        matter_defaults = MatterDefaults()
        
        
        self.region_list = input_dataset.REGION_LIST

        self.timestep = time_horizon.timestep
        self.data_timestep = time_horizon.data_timestep
        self.data_time_horizon = time_horizon.data_time_horizon
        self.model_time_horizon = time_horizon.model_time_horizon

        
        
        if self.timestep != self.data_timestep:
            # Interpolate Material Intensity Dictionary
            self._interpolate_material_intensity()

        self.Y = np.zeros((self.time_horizon, self.num_regions))
        self.Y[0, :] = initial_gdp  # Initialize Y with the GDP data for the first year
        self.material_intensity = initial_material_intensity
        self.dmc = np.zeros((self.time_horizon, self.num_regions))
        self.extracted_matter = np.zeros((self.time_horizon, self.num_regions))
        self.recycled_stock = np.zeros((self.time_horizon, self.num_regions))
        self.discarded_stock = np.zeros((self.time_horizon, self.num_regions))
        self.socioeconomic_stock = np.zeros((self.time_horizon, self.num_regions))
        self.socioeconomic_stock[0, :] = initial_ses
        self.waste = np.zeros((self.time_horizon, self.num_regions))
        self.material_reserves = np.zeros((self.time_horizon, self.num_regions))
        self.material_reserves[0, :] = initial_rev  # Initialize REV with given data
        self.material_resources = np.zeros((self.time_horizon, self.num_regions))
        self.material_resources[0, :] = initial_res  # Initialize RES with given data

    def run_simulation(self):
        for t in range(1, self.time_horizon):
            for r in range(self.num_regions):
                self.Y[t, r] = self.Y[t-1, r] * (1 + self.params.growth_rate_gdp)
                self.dmc[t, r] = self.material_intensity[r] * self.Y[t, r]
                self.discarded_stock[t, r] = self.params.proportion_of_discarded_stock * self.socioeconomic_stock[t-1, r]
                self.recycled_stock[t, r] = self.params.recycling_rate * self.discarded_stock[t, r]
                self.extracted_matter[t, r] = self.dmc[t, r] - self.recycled_stock[t, r]
                self.socioeconomic_stock[t, r] = self.socioeconomic_stock[t-1, r] + self.dmc[t, r] - self.discarded_stock[t, r]
                self.waste[t, r] = self.discarded_stock[t, r] - self.recycled_stock[t, r]
                self.material_reserves[t, r] = self.material_reserves[t-1, r] + self.params.conversion_rate_material_reserves * self.material_resources[t-1, r] - self.extracted_matter[t, r]
                self.material_resources[t, r] = self.material_resources[t-1, r] - self.material_reserves[t, r] + self.extracted_matter[t, r]

    def plot_results(self):
  

# Example usage
defaults = MatterDefaults()
matter_use = MatterUse(defaults, initial_gdp)
matter_use.run_simulation()
matter_use.plot_results()


In [7]:
import h5py
import numpy as np
import copy

class MatterDefaults:
    def __init__(self, recycling_rate=0.1, proportion_of_discarded_stock=0.013, conversion_rate_material_reserves=0.0015):
        self.recycling_rate = recycling_rate
        self.proportion_of_discarded_stock = proportion_of_discarded_stock
        self.conversion_rate_material_reserves = conversion_rate_material_reserves

class MatterUse:
    def __init__(self, defaults, gdp, initial_ses, initial_rev, initial_res, file_path, data_key, num_regions=57, num_scenarios=5):
        self.params = defaults
        self.num_regions = num_regions
        self.num_scenarios = num_scenarios
        self.num_time_steps = gdp.shape[1]  # Assuming 58 time steps
        self.gdp = gdp
        self.material_intensity_array = self.load_hdf5_data(file_path, data_key)
        self.extracted_matter = np.zeros_like(gdp)  # (57, 58, 5)
        self.socioeconomic_stock = np.zeros_like(gdp)
        self.socioeconomic_stock[:, 0, :] = initial_ses  # Initialize for the first year in all scenarios
        self.material_reserves = np.zeros_like(gdp)
        self.material_reserves[:, 0, :] = initial_rev
        self.material_resources = np.zeros_like(gdp)
        self.material_resources[:, 0, :] = initial_res

    def load_hdf5_data(self, file_path, data_key):
        with h5py.File(file_path, 'r') as file:
            return copy.deepcopy(file[data_key][:])

    def run_simulation(self):
        for s in range(self.num_scenarios):
            for t in range(1, self.num_time_steps):
                for r in range(self.num_regions):
                    current_gdp = self.gdp[r, t, s]
                    current_intensity = self.material_intensity_array[r, t, s]
                    self.extracted_matter[r, t, s] = current_intensity * current_gdp  # Simplified example
                    # Include other computations and scenario-specific logic here

    def plot_results(self):
        time_steps = np.arange(2015, 2300, 5)
        fig, ax = plt.subplots(figsize=(10, 6))
        for s in range(self.num_scenarios):
            scenario_data = np.mean(self.extracted_matter[:, :, s], axis=0)  # Average over regions
            ax.plot(time_steps, scenario_data, label=f'SSP{s+1}')
        
        ax.set_title('Extracted Matter Across Scenarios')
        ax.set_xlabel('Year')
        ax.set_ylabel('Extracted Matter (average per region)')
        ax.legend()
        plt.show()

# Example usage
file_path = 'JUSTICE\data\input\material_intensity_array.hdf5'
data_key = 'material_intensity_array'
defaults = MatterDefaults()
initial_gdp = np.random.rand(57, 58, 5)  # Replace with actual data
initial_ses = np.random.rand(57, 5)  # Replace with actual data
initial_rev = np.random.rand(57, 5)  # Replace with actual data
initial_res = np.random.rand(57, 5)  # Replace with actual data

matter_use = MatterUse(defaults, initial_gdp, initial_ses, initial_rev, initial_res, file_path, data_key)
matter_use.run_simulation()
matter_use.plot_results()


IndexError: index 18 is out of bounds for axis 1 with size 18